## 🧬 **Adatok és kísérleti protokollok**

* A sejteket **H1 hESC** vonalból származtatják, **CRISPRi dCas9-KRAB** rendszerrel, 300 gén célzással:

  > “Parental H1 hESC cells were obtained from Wi-cell ... dual guide design from Replogle et al. [1] ... 10x Genomics' Flex chemistry and sequenced on NovaSeq X … ~93k reads/cell.”
  > 👉 forrás: [Replogle et al. 2022, *eLife*](https://elifesciences.org/articles/81856)

* A „control” sejtek a **"non-targeting"** génekhez tartoznak:

  > “There are a little over 38k cells that are ‘non-targeting’.”

* Nincs külön control dataset; a non-targeting sejtekből kell ezeket kiválasztani.

---

## ⚙️ **Adatelőkészítés és normalizálás**

* A szervezők **50k UMI-re normalizáltak** és **log1p transformot** alkalmaztak:

  > “The support set data was normalized to slightly lower than the median read depth … with `scanpy.pp.normalize_total` … and also `log1p`.”
  > “For the missing columns … imputed in 0s.”

* A **validációs és teszt** készletek mediánjából származtatott normalizálás történik:

  > “For validation and test datasets, the normalization is performed with the median of the corresponding set …”

* Tehát: minden cella ≈ 10k–50k normalizált összeggel rendelkezik; a gene set 18,080 gént tartalmaz, a hiányzóakat 0-val pótolták.

---

## 🧠 **ESM2 embeddingek**

* A `ESM2_pert_features.pt` fájl létrehozása így történt:

  > “Gene embeddings were computed using ESM-2 (esm2_t48_15B_UR50D) … averaged per transcript, then per gene.”

Ez tehát a **perturbált gének fehérjeszekvenciáinak embeddingjeit** tartalmazza, amit UniProt-ból vettek, és ESM2-vel átlagoztak — ezt a fájlt nem kell újragenerálni, de reprodukálható.

---

## 📊 **Training és compute**

* A STATE modell:

  > “Trained on 14,420 AnnData files spanning 167M human cells … 4 nodes × 8 H100 GPUs … Flash Attention 2 … mixed precision bf16.”

* A Colab notebook 50k step ≈ 2 óra RTX 5090-en, és **sokkal lassabb T4 GPU-n**:

  > “training for ~50k steps took ~2 hours for me on a 5090 … expect nb training without a more powerful GPU to take a while.”

---

## 🧮 **Értékelés és cell-eval működés**

* A leaderboard három metrikát használ:

  * **MAE**

  * **DES (Differential Expression Score)** = “overlap_at_N”

  * **PDS (Perturbation Discrimination Score)** = “l1 distance”

  > “For DE Score it is ‘overlap_at_N’ in cell-eval … you can run `cell-eval run {...} --profile vcc` to reproduce leaderboard metrics.”

* A résztvevők szerint:

  > “cell-eval abstracts pdex that abstracts custom rust code … metrics should be minimalist and hackable.”

* Többen észrevették, hogy a leaderboard és saját számításaik **nem egyeznek** (különböző DE gene counts), valószínűleg a **Benjamini–Hochberg korrekció** miatt:

  > “As you add more p-values … it becomes harder to find DEGs … but if p-values are small, thresholds increase … every perturbation impacts every perturbation.”

  Ez tehát egy ismert, a több-teszt korrekcióból eredő torzítás: ha az összes gén-perturbációt együttesen tesztálod, nem páronként, a DEGs száma mesterségesen nő.

---

## 💻 **Technikai tippek**

* A `cell-eval` és `state` kód használatához:

  > “make sure in your pyproject.toml, torch is 2.7.0+cu128 or 2.7.1 works too.”

* **RTX 5090** hibánál:

  > “RuntimeError: no kernel image is available … → use torch 2.7.0+cu128.”

* A checkpointokból való visszatöltés automatikus:

  > “the model should automatically restart from the ‘last.ckpt’ found in the checkpoints folder.”

---

## 🧩 **Biológiai értelmezések**

* A DE genes kategóriák: 0–10, 10–100, 100–10 000.
  Néhányan nem találtak 0–100 DEG-et, amire az magyarázat:

  > “They filtered out negligible-effect perturbations when selecting 300 genes.”

* A kérdés: “cell line” ≠ “cell type” — batch-effect kontrollhoz használt kategória:

  > “They controlled cell line (batch effect optional) and created paired data.”

---

## ☁️ **Compute támogatás**

* Foundry (mlfoundry.com) biztosít GPU hozzáférést:

  > “Contact [arc@mlfoundry.com](mailto:arc@mlfoundry.com) … the Virtual Cell Challenge isn’t involved in allocating compute.”

---

## 🚧 **Platform hibák**

* Tipikus hibák:

  * `"convert_indices_from_csr_to_coo_cpu" not implemented for 'Float'`
    → ismert bug, GitHubon jelentve.
  * `"Missing cell-eval watermark"` → újra futtatni a `cell-eval prep` toolt.
  * `"inappropriate language"` → fájlnévben szereplő “testik” miatt 😅
  * E-mail-login bug: support tickethez jelszó kell → “FreshDesk login, not VCC.”

---

## 📚 **Összegzés – ami neked hasznos lehet**

**Kiemelten fontos pontok számodra (mint kutató és modellező):**

1. **Adatelőkészítés:** `scanpy.pp.normalize_total(..., target_sum≈50000)` + `sc.pp.log1p`; hiányzó gének → 0, subset 200 target.
2. **ESM2 features:** `esm2_t48_15B_UR50D`, per-transcript átlag → gene-szintű embedding.
3. **cell-eval profil:** `--profile vcc` használata biztosan a leaderboard-szerinti kiértékelést adja.
4. **DEG számítás:** páronkénti Wilcoxon + Benjamini–Hochberg korrekció → fontos, ha saját metrikát validálsz.
5. **Compute igény:** T4 GPU-n akár 6–8× lassabb; A100 ajánlott.
6. **Normalization consistency:** minden készlet hasonló UMI-eloszlású, de a median UMI-t a saját setből számolják.